In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
from matplotlib import pyplot as plt
import gmshtranslator.gmshtranslator as gmsh
import arrayfire as af

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import lagrange
from dg_maxwell import wave_equation_2d as w2d
from dg_maxwell import isoparam
from dg_maxwell import utils

af.set_backend(params.backend)

gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [2]:
nodes, elements = msh_parser.read_order_2_msh('read_and_plot_mesh/mesh/circle.msh')
nodes = af.np_to_af_array(nodes)
elements = af.np_to_af_array(elements)

gmshTranslator: Initializing...
gmshTranslator: Mesh has 25 nodes.
gmshTranslator: Mesh has 25 elements.
gmshTranslator: Processed 54 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements


In [3]:
N_LGL = 8
xi_LGL  = (lagrange.LGL_points(N_LGL))
eta_LGL = (lagrange.LGL_points(N_LGL))

xi, eta = utils.af_meshgrid(xi_LGL, eta_LGL)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [4]:
element_nodes = af.moddims(nodes[elements], d0 = elements.shape[0],
                           d1 = elements.shape[1], d2 = 2)
# element_nodes = af.reorder(element_nodes, d0 = 1, d1 = 2, d2 = 0)

In [5]:
print(elements)
print(element_nodes)

arrayfire.Array()
Type: long int

[4 9 1 1]
         1         10          4         18          3          9          2         17         21 
         3         18          4         11          6         19          5         12         22 
         5         19          6         13          8         20          7         14         23 
         7         16          2         17          1         15          8         20         24 


arrayfire.Array()
Type: double

[4 9 2 1]
   -0.0000    -2.1213    -3.0000    -2.0000    -1.0000    -0.7071    -0.0000     0.0000    -1.4142 
   -1.0000    -2.0000    -3.0000    -2.1213    -0.0000     0.0000    -0.0000    -0.7071    -1.4142 
   -0.0000     0.0000    -0.0000     2.1213     3.0000     2.0000     1.0000     0.7071     1.4142 
    1.0000     0.7071    -0.0000     0.0000    -0.0000     2.1213     3.0000     2.0000     1.4142 

    3.0000     2.1213     0.0000     0.0000     0.0000     0.7071     1.0000     2.0000     1.4142 
    0.0000 

In [15]:
isoparam.isoparam_x_2D(np.array(element_nodes[0, :-1, 0]), 0, 0)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [6]:
N = np.zeros([8, xi.shape[0], xi.shape[1]])
N = af.np_to_af_array(N)
N[0] = af.reorder((-1.0 / 4.0) * (1 - xi)  * (1 + eta) * (1 + xi - eta), d0 = 2, d1 = 0, d2 = 1)
N[1] = af.reorder((1.0 / 2.0)  * (1 - xi)  * (1 - eta**2), d0 = 2, d1 = 0, d2 = 1)
N[2] = af.reorder((-1.0 / 4.0) * (1 - xi)  * (1 - eta) * (1 + xi + eta), d0 = 2, d1 = 0, d2 = 1)
N[3] = af.reorder((1.0 / 2.0)  * (1 - eta) * (1 - xi**2), d0 = 2, d1 = 0, d2 = 1)
N[4] = af.reorder((-1.0 / 4.0) * (1 + xi)  * (1 - eta) * (1 - xi + eta), d0 = 2, d1 = 0, d2 = 1)
N[5] = af.reorder((1.0 / 2.0)  * (1 + xi)  * (1 - eta**2), d0 = 2, d1 = 0, d2 = 1)
N[6] = af.reorder((-1.0 / 4.0) * (1 + xi)  * (1 + eta) * (1 - xi - eta), d0 = 2, d1 = 0, d2 = 1)
N[7] = af.reorder((1.0 / 2.0)  * (1 + eta) * (1 - xi**2), d0 = 2, d1 = 0, d2 = 1)

gmshTranslator: Ending


In [7]:
N = af.reorder(N, d0 = 3, d1 = 0, d2 = 1, d3 = 2)
N = af.tile(N, d0 = element_nodes.shape[2])
N

arrayfire.Array()
Type: double
Dims: (2, 8, 8, 8)

In [13]:
af.broadcast(utils.multiply,
             N,
             element_nodes[:, : -1, 0])

RuntimeError: In function af::dim4 getOutDims(const af::dim4&, const af::dim4&, bool)
In file src/backend/ArrayInfo.cpp:180
Invalid dimension for argument 1
Expected: ldims == rdims


In [9]:
af.reorder(af.sum(af.broadcast(utils.multiply,
                               element_nodes[:, : -1, 0],
                               N),
                  dim = 1),
           d0 = 0, d1 = 2, d2 = 3, d3 = 1)

RuntimeError: In function af::dim4 getOutDims(const af::dim4&, const af::dim4&, bool)
In file src/backend/ArrayInfo.cpp:180
Invalid dimension for argument 1
Expected: ldims == rdims


In [30]:
print(element_nodes[:, : -1, 0])

arrayfire.Array()
Type: double

[4 8 1 1]
   -0.0000    -2.1213    -3.0000    -2.0000    -1.0000    -0.7071    -0.0000     0.0000 
   -1.0000    -2.0000    -3.0000    -2.1213    -0.0000     0.0000    -0.0000    -0.7071 
   -0.0000     0.0000    -0.0000     2.1213     3.0000     2.0000     1.0000     0.7071 
    1.0000     0.7071    -0.0000     0.0000    -0.0000     2.1213     3.0000     2.0000 




In [36]:
isoparam.isoparam_x_2D(nodes[elements[0], 0, xi])

TypeError: isoparam_x_2D() missing 2 required positional arguments: 'xi' and 'eta'

In [37]:
nodes[elements[0], 0]

arrayfire.Array()
Type: double
Dims: (9,)